### This file is used to create a normalized vector for all customers from a csv containing orders. The vector represents that customers preference in cuisines/food types

##### Input: Csv file containing skip the dishes orders (with customer_id, restaraunt_id and itemlist). Csv file containing restaraunt cuisines, created from AssignRestarauntCuisines.ipynb
##### Output: Csv file where each row is a specific customers preference for food, one column being the customers id and the rest being the preference for a specific cuisine from 0.0-1.0. Row is normalized so it adds up to 1.0.

In [1]:
import pandas as pd 
import numpy as np 
import math

In [2]:
# df is order csv
df = pd.read_csv("./from_skip_files/orders_11mil.csv")
# rf is restaraunt cuisines. 
rf = pd.read_csv('./derived_files/Cuisine_Restaraunt.csv')

In [3]:
print(df.shape)

(11067589, 15)


In [4]:
# List of ethnic or cultural cuisines
# Plus convenience
ethnic_or_cultural_items = [
    "Convenience",
    "Indian",
    "Italian",
    "Chinese",
    "Vietnamese",
    "Japanese",
    "Mediterranean",
    "Middle Eastern",
    "Mexican",
    "Korean",
    "Thai",
    "French",
    "African",
    "Latin American",
    "Ethiopian",
    "Caribbean",
    "Filipino",
    "Spanish",
    "German",
]

In [5]:
# Map of cuisine type to keywords that fit under that cuisine
cuisine_map = {
    "Chicken" : ["chicken"],
    "Fries" : ["fries"],
    "Beef" : ["beef"],
    "Pork" : ["pork", "bacon", "pepperoni"],
    "Rice" : ["rice"],
    "Lamb" : ["lamb"],
    "Vegetarian" : ["vegan", "vegetarian", "veggie", "beyond meat"],
    "Sandwiches & Subs" : ["sandwich", "sub", "wrap", "blt"],
    "Desserts" : ["blizzard", "ice cream", "frozen", "dessert", "chocolate", "drizzle", "desserts", "milkshake", "candy", 
                  "candies", "sundae", "oreo", "skor", "brownie", "shake", "tiramisu", "timbits", "ben and jerry", 
                  "cheesecake", "cookie"],
    "Canadian" : ["canadian", "canadien", "alberta","poutine"],
    "Fast food" : ["fast food","combo", "meal"],
    "Burgers" : ["burger", "patty", "mcdouble", "big mac", "quarter pounder"],
    "Seafood" : ["fish", "seafood", "shrimp", "crab", "lobster", "prawn", "seaweed", 
                 "salmon", "tuna", "poke", "calamari", "squid", "fish and chips"],
    "Healthy" : ["organic", "health", "protein", "salad", "fresh", "tofu",  "fruit", "water", "vegetable", "smoothie", "parfait"],
    "Pizza" : ["pizza"],
    "Breakfast & Brunch" : ["egg", "toast", "benedict", "breakfast", "brunch", "cereal", "pancake", "waffle", "hash brown"],
    "Coffee/Tea" : ["coffee", " tea ", "americano", "cappuccino", "latte", "cafe", "chai", "london fog"],
    "Alcohol" : ["beer", "wine", "liquor", "budweiser", "bud light", "spirits", "corona", "stella artois", 
                 "michelob ultra", "mike's hard", "labatt", "sauvignon", "smirnoff", "vodka", "whisky", "cognac",
                 "white claw", "pinot noir"],
    "Noodles" : ["noodle", "vermicelli"],
    "Pub food" : ["wing", "onion ring", "wedge", "mac & cheese", "mac and cheese", "gravy", "mashed potato", "breadsticks"], 
    "Indian" : ["indian", "naan", "nan", "pakora", "samosa", "masala", "aloo", "paneer", "biryani", "tandoori", "roti", "tikka"],
    "Italian" : ["italian", "pasta", "spaghetti", "penne", "fettuccini", "lasagna", "lasagne", "linguini", "ravioli", "tortellini", "meatball", "canoli"],
    "Bakery" : ["danish", "cake", "bun", "donut", "muffin","bagel", "doughnut", "pie", "scone", "cinnamon roll", "loaf"],
    "Barbecue" : ["barbecue", "bbq", "grill", "buffalo"],
    "Chinese" : ["chinese", "china", "hot pot", "wonton", "cantonese", "mein", "gyoza", "dumpling"],
    "Vietnamese" : ["vietnamese", "pho", "viet", "bun cha","ca kho to"],
    "Japanese" : ["japanese","japan", "ramen", "sashimi", "teriyaki", "katsu", "tempura", "edamame", "bento", "takoyaki"],
    "Tacos": ["taco"],
    "Sushi" : ["sushi"],
    "Mediterranean" : ["mediterranean","pita", "damascus", "greek", "greece", "briam","taramasalata", "opa"],
    "Hot Dogs & Sausages" : ["hot dogs","sausage", "weiner"],
    "Middle Eastern" : ["middle eastern","falafel", "hummus","shawarma", "baklava", "donair", "tzatziki"],
    "Convenience" : ["convenience","pre-made","grocery", "slurpee"],
    "Mexican" : ["mexican", "chilaquiles", "burrito", "nacho", "quesadilla", "queso", "taquito", "salsa"],
    "Steakhouse" : ["steakhouse", "steak"],
    "Halal" : ["halal", "zabiha"],
    "Korean" : ["korean", "kimchi", "bulgogi","bibimbap","tteokbokki","jjambbong","doenjang"],
    "Thai" : ["thai", "tom yum goong", "green curry"],
    "Soup" : ["soup"],
    "Gluten Free" : ["gluten free", "no gluten"],
    "Popcorn": ["popcorn"], 
    "Pet Food" : ["pet", "dog", "cat"],
    "Bubble Tea" : ["bubble tea", "boba", "milk tea", "taro milk"],
    "French" : ["french", "francais", "crepe", "foie gras","coq au vin","cassoulet","baguette","croissant","gougeres","cajun & creole", "creole"],
    "African" : ["african","pap en vleis", "shisa nyama","bunny chow","koshari"],
    "Latin American" : ["latin","asado","saltena","feijoada","empanada","bandeja paisa",
                        "gallo pinto","ropa vieja","mangu","encebollado", "pupusas","pepian","peruvian"],
    "Haute Cuisine" : ["haute", "high class", "expensive", "champagne"],
    "Ethiopian" : ["ethiopian","tibs","kitfo","beyainatu","fuul"],
    "Caribbean" : ["caribbean", "jamaica","barbados","bahamas"],
    "Filipino" : ["filipino","adobo","lechon","sisig","bulalo"],
    "Spanish" : ["spanish","paella valenciana","patatas bravas","gazpacho","pimientos de padron","jamon","tapas","churro"],
    "Butcher" : ["raw", "butcher","delicatessen"],
    "Kosher" : ["kosher", 'kashrut', 'jewish'],
    "German" : ["german","schnitzel","rouladen","eintopf","sauerbraten"]
}
print(len(list(cuisine_map.keys())))

54


In [6]:
# Checks for any repeats in value lists accross cuisine types in the map above
freq = {}
for key in cuisine_map.keys():
    for food in cuisine_map[key]:
        if food in freq:
            freq[food] += 1    
        else:
            freq[food] = 1
for key in freq.keys():
    if freq[key] > 1:
        print(key)

In [7]:
# # Calculates the amount of items that the cusine map catches
# hit = 0
# all = 0
# test = []
# test2 = []
# def f(x, y):
#   global hit, all
#   ret_val = False
#   res = isinstance(x, str)
#   if not res:
#     print(x)
#     return False
#   items = x.split(", ")
#   any_items_found = False
#   for item in items:
#     item_found = False
#     all +=1
#     temp_item = item.lower()
#     flag = False
#     for key in cuisine_map:
#       if flag: 
#         break
#       for food in cuisine_map[key]:
#         if food in temp_item:
#           hit += 1
#           ret_val = True
#           any_items_found = True
#           flag = True
#           item_found = True
#           break
#     if not item_found:
#       test.append(temp_item)
#       test2.append(y) 
#           # return True
#   # if not any_items_found:
#   #   r_name = y.lower()
#   #   flag = False
#   #   for key in cuisine_map:
#   #     if flag:
#   #       break
#   #     for food in cuisine_map[key]:
#   #       if food in r_name:
#   #         hit +=1 
#   #         flag = True
#   #         ret_val = True
#   #         break
#   return ret_val

# result = [f(x,y) for x,y in zip(df['item_list'],df['restaurant_short_name'])]
# print(sum(result))
# print(len(result))
# print(hit)
# print(all)
# print(hit/all*100)

In [8]:
print(df['customer_id'].value_counts().describe())
orders_by_customer = df.groupby('customer_id').size().reset_index(name='order_count')
customers_with_many_orders = orders_by_customer[orders_by_customer['order_count'] > 10]
print(customers_with_many_orders['customer_id'].shape)


count    567134.000000
mean         19.514945
std          39.309175
min           1.000000
25%           1.000000
50%           5.000000
75%          20.000000
max        6223.000000
Name: customer_id, dtype: float64
(205077,)


In [9]:
import datetime
import pytz
import dateutil.parser

#This function takes in the order date, and returns weight based on the recency of the order
#Under 3 months it has weight of 1
#Between 3 months to 6 months, it has weight between 1 to 0.7
#Between 6 months to 12 months, it has weight between 0.7 to 0.3
#Between 24 months to 12 months, it has weight between 0.3 to 0.0
#After 24 months the weight is 0, meaning we don't consider orders after 2 years from now

def get_order_weight(order_date_str):
    order_date = dateutil.parser.parse(order_date_str)
    delta = datetime.datetime.now(pytz.utc) - order_date
    days_since_order = delta.days
    weight_0to3months = 1
    weight_3to6months = 0.7
    weight_6to12months = 0.3
    weight_12to24months = 0.2
    if days_since_order < 90:
        return weight_0to3months
    elif days_since_order < 180:
        slope = (weight_3to6months - weight_0to3months) / (180 - 90)
        return 1.0 + slope * (days_since_order - 90)
    elif days_since_order < 365:
        slope = (weight_6to12months - weight_3to6months) / (365 - 180)
        return 0.7 + slope * (days_since_order - 180)
    elif days_since_order < 730:
        slope = (weight_12to24months - weight_6to12months) / (730 - 365)
        return 0.3 + slope * (days_since_order - 365)
    else:
        return 0.1

In [10]:
# Main method that counts frequency of cuisine types occuring in users order history and uses that
# as a metric to calculate their preference by dividing that specific cuisine by total cuisines
# if a user orders an item that has the word "chicken" in it 3 times and had 20 items total their
# prefence for chicken would be 3/20 or 0.15.
#Instead of doing cuisine + 1, for every cuisine matched, this code now does cuisine + weight. With weight, it will
#It will now take into account of the recency of the order
overall_cuisine = dict(zip(cuisine_map.keys(), [0] * 80))
grouped = df.groupby('customer_id')
customer_profile_list = []
miss = 0
def process_group(group):
    global overall_cuisine
    global miss 
    customer_profile = dict(zip(cuisine_map.keys(), [0] * 80))
    total_filtered = 0
    for index, row in group.iterrows():
        items = row['item_list'].split(", ")
        order_date = row['created_time']
        weight = get_order_weight(order_date)
        for item in items:
            lower_item = item.lower()
            cuisine_identified_item = False
            item_found = False
            for key in cuisine_map:
                for cuisine in cuisine_map[key]:
                    if cuisine in lower_item:    
                        if key in ethnic_or_cultural_items:
                            cuisine_identified_item = True  
                        customer_profile[key] += weight
                        overall_cuisine[key] += weight
                        total_filtered += weight
                        item_found = True
                        break
            if not cuisine_identified_item:
                restaraunt_cuisines_df = rf[rf['short_name'] == row['restaurant_short_name']]['cussine_list']
                if restaraunt_cuisines_df.shape[0] > 0:
                    restaraunt_cuisines_list = restaraunt_cuisines_df.str.split(", ")
                    restaraunt_cuisines = restaraunt_cuisines_list.iloc[0]
                    for cuisine in restaraunt_cuisines:
                        customer_profile[cuisine] +=weight 
                        overall_cuisine[cuisine] +=weight
                        total_filtered += weight
    map_total_filtered = 0
    for value in customer_profile.values():
        map_total_filtered += value
    if math.isclose(total_filtered, map_total_filtered, abs_tol=0.003) and total_filtered != 0:
        for key in customer_profile:
            customer_profile[key] = customer_profile[key]/total_filtered
        customer_profile['customer_id'] = group.name
        customer_profile_list.append(customer_profile)
    else:
        miss += 1
#         print(map_total_filtered, " ", total_filtered)
#         print("missed customer: ", group.name)
    if len(customer_profile_list) % 5000 == 0:
        print(len(customer_profile_list)/567134) 
        print(miss)


grouped.apply(process_group)

0.008816258591443997
104
0.017632517182887993
210
0.02644877577433199
315
0.03526503436577599
431
0.044081292957219985
537
0.05289755154866398
655
0.06171381014010798
764
0.06171381014010798
765
0.07053006873155197
881
0.07934632732299597
993
0.08816258591443997
1115
0.09697884450588397
1241
0.10579510309732797
1377
0.11461136168877196
1479
0.12342762028021596
1592
0.13224387887165995
1706
0.14106013746310395
1817
0.14987639605454794
1907
0.15869265464599194
2021
0.16750891323743594
2136
0.17632517182887994
2240
0.18514143042032394
2339
0.19395768901176794
2435
0.20277394760321193
2552
0.21159020619465593
2672
0.22040646478609993
2792
0.22922272337754393
2903
0.23803898196898793
3003
0.24685524056043193
3117
0.2556714991518759
3236
0.2644877577433199
3331
0.2733040163347639
3425
0.2821202749262079
3533
0.2909365335176519
3638
0.2997527921090959
3745
0.3085690507005399
3854
0.3173853092919839
3963
0.3262015678834279
4051
0.3350178264748719
4167
0.3438340850663159
4268
0.3526503436577599

""


In [11]:
sorted_dict = sorted(overall_cuisine.items(), key=lambda x: x[1], reverse=True)
for item in enumerate(sorted_dict):
    print(item[1])

('Chicken', 2690142.2954843403)
('Fast food', 2140532.6498598387)
('Burgers', 1691536.8238081832)
('Sandwiches & Subs', 1312594.4885521063)
('Bakery', 1200937.10365888)
('Pork', 1067167.0020941237)
('Desserts', 958886.2524420838)
('Beef', 936210.6366738789)
('Breakfast & Brunch', 848300.095966765)
('Healthy', 820202.3042315608)
('Seafood', 754315.5453126938)
('Pizza', 749066.1053041304)
('Fries', 635720.7084473346)
('Coffee/Tea', 556082.500254059)
('Japanese', 502399.8176303782)
('Barbecue', 463240.0485004567)
('Noodles', 459198.68385284505)
('Pub food', 454420.2368504388)
('Rice', 440324.52756007813)
('Indian', 439126.1585066493)
('Middle Eastern', 366434.5862445446)
('Mexican', 358714.09242498473)
('Soup', 326628.50830431713)
('Chinese', 279823.79270514793)
('Canadian', 278947.48402811436)
('Hot Dogs & Sausages', 251649.50526100572)
('Mediterranean', 206183.55005675208)
('Italian', 203274.73012833204)
('Butcher', 187484.88988274528)
('Vegetarian', 177760.05813648255)
('Alcohol', 1613

In [12]:
# Checks how many customers which the filter couldn't match any of their items so they have no 
# information on their preferences.
customer_profile_df = pd.DataFrame(customer_profile_list)
print(df['customer_id'].nunique())
print(customer_profile_df['customer_id'].nunique())

567134
554785


In [13]:
# Reorganizes the columns so id appears first and cuisines are sorted in alphabetical order
customer_profile_df = customer_profile_df.reindex(columns=['customer_id'] + list(customer_profile_df.columns.drop('customer_id')))
cols = customer_profile_df.columns.tolist()
cols = cols[:-54] + sorted(cols[-54:])
customer_profile_df = customer_profile_df.reindex(columns=cols)
print(customer_profile_df)

                                 customer_id  African   Alcohol    Bakery  \
0       0000000d-9a20-4580-85aa-f4ca9062388c      0.0  0.000000  0.000000   
1       0000267e-c83e-4a48-9776-8163eab97b6a      0.0  0.000000  0.010936   
2       00007e2c-cd9c-43d5-9148-60457215dfd8      0.0  0.000000  0.085450   
3       00008ec4-df06-4b68-a809-b043a65faf7e      0.0  0.000000  0.000000   
4       0000a630-1c0a-45db-8516-d07bda7f22dd      0.0  0.000000  0.000000   
...                                      ...      ...       ...       ...   
554780  ffff3fd7-6d6b-4c1a-9eb1-c84c8adb5b09      0.0  0.000000  0.000000   
554781  ffff7355-c829-4fd8-b8f9-4f4a33054ded      0.0  0.000000  0.000000   
554782  ffff8464-04e3-4dbc-91f8-264c82c0784b      0.0  0.000000  0.023492   
554783  ffffe79c-5b65-4eeb-a9b1-e6af019628b7      0.0  0.045828  0.000000   
554784  fffffaec-802b-468f-b621-4f2d923aa9a0      0.0  0.241962  0.050495   

        Barbecue      Beef  Breakfast & Brunch  Bubble Tea   Burgers  \
0  

In [14]:
# Make sure each row adds in table adds to 1, since each row is normalized this should always happen.
for index, row in customer_profile_df.iterrows():
    if row[1:].sum() > 1.05 or row[1:].sum() < 0.95:
      print(row[1:].sum())
      print("Something is wrong at index: ", index)

In [15]:
# Converts to CSV file
print(customer_profile_df)
customer_profile_df.to_csv('./derived_files/customer_profiles_recency.csv', index=False)

                                 customer_id  African   Alcohol    Bakery  \
0       0000000d-9a20-4580-85aa-f4ca9062388c      0.0  0.000000  0.000000   
1       0000267e-c83e-4a48-9776-8163eab97b6a      0.0  0.000000  0.010936   
2       00007e2c-cd9c-43d5-9148-60457215dfd8      0.0  0.000000  0.085450   
3       00008ec4-df06-4b68-a809-b043a65faf7e      0.0  0.000000  0.000000   
4       0000a630-1c0a-45db-8516-d07bda7f22dd      0.0  0.000000  0.000000   
...                                      ...      ...       ...       ...   
554780  ffff3fd7-6d6b-4c1a-9eb1-c84c8adb5b09      0.0  0.000000  0.000000   
554781  ffff7355-c829-4fd8-b8f9-4f4a33054ded      0.0  0.000000  0.000000   
554782  ffff8464-04e3-4dbc-91f8-264c82c0784b      0.0  0.000000  0.023492   
554783  ffffe79c-5b65-4eeb-a9b1-e6af019628b7      0.0  0.045828  0.000000   
554784  fffffaec-802b-468f-b621-4f2d923aa9a0      0.0  0.241962  0.050495   

        Barbecue      Beef  Breakfast & Brunch  Bubble Tea   Burgers  \
0  